In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
from src.learning.training.training_transformer import TrainingTransformer
from src.learning.training.label_collector import LabelCollector
from src.learning.training.training_file_reader import TrainingFileReader
from src.learning.models import create_cnn, create_mlp, create_multi_model

config_manager = ConfigurationManager()
config = config_manager.config

reader = TrainingFileReader(path_to_training='../../training/laps/')
transformer = TrainingTransformer(config)
collector = LabelCollector()

def memory_creator(new_frame, memory, length=4, interval=2):
    memory.append(new_frame)
    
    near_memory = memory[::-interval]
    if len(near_memory) < length:
        return None
    
    if len(memory) >= length * interval:
        memory.pop(0)
        
    return np.concatenate(near_memory, axis=2)

filenames = ['lap_5_2020_01_24', 'lap_6_2020_01_24', 'lap_7_2020_01_24']

In [2]:
def read_stored_data(filename):
    df = reader.read_telemetry_as_csv(filename + '.csv')
    telemetry = collector.collect_numeric_inputs(df)
    diffs = collector.collect_expert_labels(df)

    frames = reader.read_training_video(filename + '.avi')
    resized_frames_np = transformer.resize_video_for_training(frames)

    #telemetry_np = telemetry.to_numpy()
    #diffs_np = diffs.to_numpy()
    
    return resized_frames_np, telemetry, diffs

#frames_train, frames_test, telemetry_train, telemetry_test, diffs_train, diffs_test = train_test_split(resized_frames_np, telemetry_np, diffs_np, test_size=0.2)

In [3]:
def prep_mem_datas(frames, telemetry, diffs, length, interval):
    mem_telemetry = telemetry
    mem_diffs = diffs
    
    memory = []
    mem_frames = []
    for frame in frames:
        mem_frame = memory_creator(frame, memory, length=length, interval=interval)

        if mem_frame is not None:
            mem_frames.append(mem_frame)

    mem_frames_np = np.array(mem_frames)

    len_diff = mem_telemetry.shape[0] - mem_frames_np.shape[0]
    if len_diff > 0:
        mem_telemetry = mem_telemetry.iloc[len_diff:]
        mem_diffs = mem_diffs.iloc[len_diff:]
    
    mem_telemetry_np = mem_telemetry.to_numpy()
    mem_diffs_np = mem_diffs.to_numpy()

    return mem_frames_np, mem_telemetry_np, mem_diffs_np

In [4]:
experiments = [(1, 1), (4, 1), (4, 4), (4, 8), (4, 12), (6, 4), (6, 8), (6, 12)]
datas_list = []

for experiment in tqdm(experiments):
    frames_mem = None
    telemetry_mem = None
    diffs_mem = None
    
    for training_file in filenames:
        frames_i, telem_i, diffs_i = prep_mem_datas(*read_stored_data(training_file), *experiment)
        
        if frames_mem is None and telemetry_mem is None and diffs_mem is None:
            frames_mem = frames_i
            telemetry_mem = telem_i
            diffs_mem = diffs_i
        else:
            frames_mem = np.append(frames_mem, frames_i, axis=0)
            telemetry_mem = np.append(telemetry_mem, telem_i, axis=0)
            diffs_mem = np.append(diffs_mem, diffs_i, axis=0)
        
    datas_list.append((frames_mem, telemetry_mem, diffs_mem))

In [ ]:
epochs = 6
batch_size = 32
verbose = 0

results = {}

for data in tqdm(datas_list):
    result_desc = "N={}, M={}".format(*experiments[datas_list.index(data)])
    tqdm.write(result_desc)
    
    frames_train, frames_test, telemetry_train, telemetry_test, diffs_train, diffs_test = train_test_split(*data, test_size=0.2)
    
    losses = []
    val_losses = []
    for i in tqdm(range(0, 10)):
        mlp = create_mlp()
        cnn = create_cnn(input_shape=frames_train[0].shape)
        multi = create_multi_model(mlp, cnn)
    
        hist = multi.fit([telemetry_train, frames_train], diffs_train,
                        validation_data=([telemetry_test, frames_test], diffs_test),
                        epochs=epochs, batch_size=batch_size, verbose=verbose)
        
        losses.append(hist.history['loss'])
        val_losses.append(hist.history['val_loss'])
    
    mean_losses = [np.mean(i) for i in zip(*losses)]
    mean_val_losses = [np.mean(i) for i in zip(*val_losses)]
    
    results[result_desc] = mean_losses
    results[result_desc + "_val"] = mean_val_losses
    
    tqdm.write("Mean loss per epoch: {}".format(mean_losses))
    tqdm.write("Mean validation loss per epoch: {}".format(mean_val_losses))

N=1, M=1


In [ ]:
print(results)

In [ ]:
mlp = create_mlp()
cnn = create_cnn(input_shape=frames_train[0].shape)
multi = create_multi_model(mlp, cnn)
#multi.summary()

multi.fit([telemetry_train, frames_train], diffs_train,
                validation_data=([telemetry_test, frames_test], diffs_test),
                epochs=epochs,
                batch_size=batch_size,
                verbose=verbose)